In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import json
from datetime import datetime
import pickle
import time

### Initial step: build dictionary of `{year:mainurl}`

In [ ]:
r = requests.get('https://www.letour.fr/en/history')
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all('button') #class="dateTabs__link js-tabs"')

In [ ]:
year_links_dict = {}

for item in results:
    if item.text:
        key = int(item.text)
        value = {'mainurl':item['data-tabs-ajax']}
        year_links_dict[key] = value
        
print(len(year_links_dict))
year_links_dict#[2018]

In [ ]:
for year, dct in year_links_dict.items():
    print(year, dct['mainurl'])

#### Adding links to `Starters`, `Stages`, `Jersey wearers`, `Stage winners` and `Ranking` 

In [ ]:
base_url = 'https://www.letour.fr'

for year in list(year_links_dict.keys()):
    print('Main URL: ', year, year_links_dict[year]['mainurl'])
    r = requests.get(base_url + year_links_dict[year]['mainurl'])
    soup = BeautifulSoup(r.text, 'html.parser')
    
    new_dict = {}
    buttons = soup.find_all('button', class_="js-tabs-nested")
    for b in buttons:
        new_key = b.text.lower().replace(' ', '_') + '_url'
        new_val = b['data-tabs-ajax']
        year_links_dict[year][new_key] = new_val

Save to `pickle`

In [2]:
def save_pickle(obj, filename):
    with open('data/' + filename + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data/year_links_dict.pkl', 'wb') as f:
    pickle.dump(year_links_dict, f, pickle.HIGHEST_PROTOCOL)

Open from `pickle`

In [3]:
def load_pickle(name):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
year_links_dict = load_pickle('year_links_dict')
year_links_dict

#### Add `num_of_stages` to dict

Function to scrape the total number of stages, for a particular year

In [4]:
base_url = 'https://www.letour.fr'

def scrape_number_of_stages(year):
    print(f'Scraping number of stages for {year}')
    r = requests.get(base_url + year_links_num_dict[year]['mainurl'])
    soup = BeautifulSoup(r.text, 'html.parser')
    return int(soup.find_all('span', class_='statsInfos__number')[0].text)

In [ ]:
# mainurl = '/en/block/history/11818/f34c3404d95a697dcf77d4cd8e8278fa' # 2018 - 21 stages
# mainurl = '/en/block/history/10804/89b36b01ffe439e016ec1d59c57b63d1' # 2011 - 21 stages
mainurl = '/en/block/history/10708/0b76b8f809ad5d8bcf3579df597644d8'  # 1904 - 6 stages

print(scrape_number_of_stages(1956))

In [ ]:
year_links_num_dict = year_links_dict.copy()

for year in list(year_links_num_dict.keys()):
    year_links_num_dict[year]['num_of_stages'] = scrape_number_of_stages(year)

In [ ]:
save_pickle(year_links_num_dict, 'year_links_num_dict')

### Load `year_links_num_dict`

In [5]:
year_links_num_dict = load_pickle('year_links_num_dict')
year_links_num_dict

{1903: {'mainurl': '/en/block/history/10707/d0ab6a216569236433268b7f19e1776c',
  'ranking_url': '/en/block/history/10707/ranking/c4eb59986a1ba20371a3df6c74ceb25b',
  'starters_url': '/en/block/history/10707/starters/df151b25da25a66cf629b8098211c08c',
  'the_race_url': '/en/block/history/10707/the_race/3679370d92b2c33c963ce44566d6da93',
  'stages_url': '/en/block/history/10707/stages/d2ee4f144d6a7ce9bcad5d7aada4b86d',
  'jersey_wearers_url': '/en/block/history/10707/jerseys/d92fd8558eb3e81c7e5d199a96c031eb',
  'stages_winners_url': '/en/block/history/10707/winners/fb20799446ca7516bcf7a9844bda488a',
  'num_of_stages': 6},
 1904: {'mainurl': '/en/block/history/10708/0b76b8f809ad5d8bcf3579df597644d8',
  'ranking_url': '/en/block/history/10708/ranking/357e5c608514340a1b3d2bff0fd396c7',
  'starters_url': '/en/block/history/10708/starters/7fcff3460678d7b275f79e468d02dd05',
  'the_race_url': '/en/block/history/10708/the_race/ddd82648a02bd8fcebda0a33f61b0d4a',
  'stages_url': '/en/block/history

## Define functions to scrape data from different tables

#### 1. Starters

In [6]:
base_url = 'https://www.letour.fr'

def scrape_starters(year):
    print(f'Scraping starters for {year}')
    r = requests.get(base_url + year_links_num_dict[year]['starters_url'], timeout=10)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    rows = soup.find_all('tr')
    rows_list = []

    for i, row in enumerate(rows):
        rows_list.append(row.text.strip())
        
    starters_list = []

    for row in rows_list:
        matched_on_team = re.search(r'^[a-zA-Z]', row)

        if matched_on_team: # if row is a team name
            new_row = [row,None,None]
            starters_list.append(new_row)
        else: # row is a rider
            new_row = [None]
            new_row.append([x.strip() for x in row.split('\n ')][0])
            new_row.append([x.strip() for x in row.split('\n ')][1])
            starters_list.append(new_row)
            
    starters_df = pd.DataFrame(starters_list, columns=['team','rider_num','rider_name'])
    starters_df.team = starters_df.team.fillna(method='ffill')
    starters_df = starters_df[starters_df.rider_num.isnull() == False]
    starters_df = starters_df.reset_index(drop=True)
    filepath = 'data/' + str(year) + '/' + str(year) + '_starters' + '.csv'
    starters_df.to_csv(filepath, index=False)
    print('Saved ' + filepath)    

#### 2. Stages

In [7]:
base_url = 'https://www.letour.fr'

def scrape_stages(year):
    print(f'Scraping stages for {year}')
    r = requests.get(base_url + year_links_num_dict[year]['stages_url'], timeout=10)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    stages_list = []
    rows = soup.find_all('tr')
    for i, row in enumerate(rows):
        stages_list.append(row.text.strip())

    stages_list = [i.split('\n') for i in stages_list]
    stages_list.pop(0)
    header = ['stage_num','date_start','start_city','finish_city']
    stages_df = pd.DataFrame(stages_list, columns=header)
    filepath = 'data/' + str(year) + '/' + str(year) + '_stages' + '.csv'
    stages_df.to_csv(filepath, index=False)
    print('Saved ' + filepath)    

#### 3. Jersey wearers

In [198]:
base_url = 'https://www.letour.fr'

def scrape_jersey_wearers(year):
    print(f'Scraping jersey wearers for {year}')
    r = requests.get(base_url + year_links_num_dict[year]['jersey_wearers_url'], timeout=10)
    soup = BeautifulSoup(r.text, 'html.parser')
    jersey_list = []
    num_of_cols = 5
    
    rows = soup.find_all('tr')
    for row in rows:
        new_row = row.text.strip('\n')
        new_row = re.sub(r'\s\s+',',',new_row)
        #new_row = re.sub(r',$','',new_row) # commented out to prevent column number not matching issue
        jersey_list.append(new_row)

    jersey_list = [i.split(',') for i in jersey_list]
    header = jersey_list.pop(0)
    header = [x.lower().replace(' ', '_') for x in header[0].split('\n')] # modified header code to fix col match bug
    header[0] = 'stage_num'
    #header = ['stage_num','yellow_jersey','green_jersey','polka_dot_jersey','white_jersey']
    jersey_df = pd.DataFrame(jersey_list)
    
    while len(header) > len(jersey_df.columns): # while header is longer than num of cols in df
        header.pop()
    
    while len(jersey_df.columns) > num_of_cols: # while df has more columns than it should (fixes column number not matching bug)
        jersey_df = jersey_df.drop(jersey_df.columns[len(jersey_df.columns)-1], axis=1)
    
    jersey_df.columns = header

    filepath = 'data/' + str(year) + '/' + str(year) + '_jersey_wearers' + '.csv'
    jersey_df.to_csv(filepath, index=False)
    print('Saved ' + filepath)    

#### 4. Stage winners

In [9]:
base_url = 'https://www.letour.fr'

def scrape_stage_winners(year):
    print(f'Scraping stage winners for {year}')
    r = requests.get(base_url + year_links_num_dict[year]['stages_winners_url'], timeout=10)
    soup = BeautifulSoup(r.text, 'html.parser')
    stage_winners_list = []

    rows = soup.find_all('tr')
    for row in rows:
        new_row = row.text.strip('\n')
        new_row = re.sub(r'\s\s\s+',',',new_row) # added a third \s to fix bug of splitting on team name in parens
        new_row = re.sub(r',$','',new_row)
        new_row = re.sub(r'\n',',',new_row)
        stage_winners_list.append(new_row)

    stage_winners_list = [i.split(',') for i in stage_winners_list]
    stage_winners_list.pop(0)
    header = ['stage_num','parcours','winner','team']
    stage_winners_df = pd.DataFrame(stage_winners_list
                                    , columns=header
                                   )
    filepath = 'data/' + str(year) + '/' + str(year) + '_stage_winners' + '.csv'
    stage_winners_df.to_csv(filepath, index=False)
    print('Saved ' + filepath)

#### 5. Rankings per stage (i.e. times, gaps, points, etc.)
What are `b` and `p`?

Function to loop through all ranking codes for a given year, and scrape/export data as CSVs

In [10]:
ranking_cats = {'indiv_general':'itg',
                'indiv_stage':'ite',
                'points_general':'ipg',
                #'points_stage':'ipe',
                'climber_general':'img',
                #'climber_stage':'ime',
                'youth_general':'ijg',
                #'combative_general':'icg',
                'team_stage':'ete',
                'team_general':'etg'
               }

In [11]:
base_url = 'https://www.letour.fr'

def scrape_all_rankings(year):
    print(f'Scraping rankings for all codes for {year}')
    
    for label, code in ranking_cats.items(): # loop through ranking codes
    
        print(label, code)

        num_columns = -1 # added this code block to fix header length not matching number of cols of data bug
        if code == 'itg' or code == 'ite':
#             continue
            header = ['rank','rider','rider_no','team','times','gap','b','p']
            num_columns = 8
        elif code == 'ipg':
#             continue
            header = ['rank','rider','rider_no','team','points','b','p']
            num_columns = 7
        elif code == 'img':
            header = ['rank','rider','rider_no','team','points']
            num_columns = 5
        elif code == 'ijg':
#             continue
            header = ['rank','rider','rider_no','team','times','gap']
            num_columns = 6
        elif code == 'ete' or code == 'etg':
#             continue
            header = ['rank','team','times','gap']
            num_columns = 4
        else:
            print('Not a ranking code I am interested in!')

        rankings_df = pd.DataFrame()

        for stage_num in range(year_links_num_dict[year]['num_of_stages']): # loop through stages
            stage_num += 1
            print('\nStage number:', stage_num)
            full_url = str(base_url + year_links_num_dict[year]['ranking_url'] 
                         + f"?stage={stage_num}" 
                         + f"&type={code}")
            r = requests.get(full_url, timeout=None)
            soup = BeautifulSoup(r.text, 'html.parser')
            print(full_url)

            rows_for_df = {} 
            row_num = 0
            for item in soup.tbody.find_all('tr'):
                row = item.find_all('td')

                if len(row) == num_columns: # check for if number of columns in header matches

                    new_row = []
                    for col in row: 
                        new_row.append(col.text.strip())
                        rows_for_df[row_num] = new_row
                    row_num += 1
               
            df = pd.DataFrame.from_dict(rows_for_df, orient='index'
                                    , columns=header
                                       )

            df['stage_num'] = stage_num
            cols = list(df.columns)
            cols = [cols[-1]] + cols[:-1]
            df = df[cols]
            df = df.reset_index(drop=True)

            rankings_df = pd.concat([rankings_df, df])
                                 
        filepath = 'data/' + str(year) + '/' + str(year) + '_rankings_' + code + '.csv'
        rankings_df.to_csv(filepath, index=False)
        print('Saved ' + filepath)
        time.sleep(5)

## Time to Scrape! Loop through all years...

Finished years: 

`[
1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,
2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,
2010,2011,2012,2013,2014,2015,2016,2017,2018
]`

In [203]:
2018-1966+1

53

In [204]:
list(reversed(list(year_links_num_dict.keys())))[53:]

[1965,
 1964,
 1963,
 1962,
 1961,
 1960,
 1959,
 1958,
 1957,
 1956,
 1955,
 1954,
 1953,
 1952,
 1951,
 1950,
 1949,
 1948,
 1947,
 1939,
 1938,
 1937,
 1936,
 1935,
 1934,
 1933,
 1932,
 1931,
 1930,
 1929,
 1928,
 1927,
 1926,
 1925,
 1924,
 1923,
 1922,
 1921,
 1920,
 1919,
 1914,
 1913,
 1912,
 1911,
 1910,
 1909,
 1908,
 1907,
 1906,
 1905,
 1904,
 1903]

In [161]:
base_url = 'https://www.letour.fr'

for year in list(reversed(list(year_links_num_dict.keys())))[53:]: # starting from 1965, going backwards
    directory = 'data/' + str(year)
    if not os.path.exists(directory):
        os.makedirs(directory)

    scrape_all_rankings(year) # 1. scrape rankings data
    time.sleep(10)
    scrape_starters(year) # 2. scrape starters data
    time.sleep(10)
    scrape_stages(year) # 3. scrape stages data
    time.sleep(10)
    scrape_jersey_wearers(year) # 4. scrape jersey wearers data
    time.sleep(10)
    scrape_stage_winners(year) # 5. scrape stage winners data
    time.sleep(10)

Scraping rankings for all codes for 1973
indiv_general itg

Stage number: 1
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=1&type=itg

Stage number: 2
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=2&type=itg

Stage number: 3
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=3&type=itg

Stage number: 4
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=4&type=itg

Stage number: 5
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=5&type=itg

Stage number: 6
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=6&type=itg

Stage number: 7
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=7&type=itg

Stage number: 8
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de9611

https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=7&type=img

Stage number: 8
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=8&type=img

Stage number: 9
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=9&type=img

Stage number: 10
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=10&type=img

Stage number: 11
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=11&type=img

Stage number: 12
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=12&type=img

Stage number: 13
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=13&type=img

Stage number: 14
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=14&type=img

Stage number: 15
https://www.letour.fr/en/bloc

https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=14&type=etg

Stage number: 15
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=15&type=etg

Stage number: 16
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=16&type=etg

Stage number: 17
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=17&type=etg

Stage number: 18
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=18&type=etg

Stage number: 19
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=19&type=etg

Stage number: 20
https://www.letour.fr/en/block/history/10766/ranking/3024ac51f7381c7be040d1fd7de96114?stage=20&type=etg
Saved data/1973/1973_rankings_etg.csv
Scraping starters for 1973
Saved data/1973/1973_starters.csv
Scraping stages for 1973
Saved data/1973/1973_stages.csv
Scraping

https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=18&type=ipg

Stage number: 19
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=19&type=ipg

Stage number: 20
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=20&type=ipg
Saved data/1972/1972_rankings_ipg.csv
climber_general img

Stage number: 1
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=1&type=img

Stage number: 2
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=2&type=img

Stage number: 3
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=3&type=img

Stage number: 4
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=4&type=img

Stage number: 5
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=5&typ

https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=4&type=etg

Stage number: 5
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=5&type=etg

Stage number: 6
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=6&type=etg

Stage number: 7
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=7&type=etg

Stage number: 8
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=8&type=etg

Stage number: 9
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=9&type=etg

Stage number: 10
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=10&type=etg

Stage number: 11
https://www.letour.fr/en/block/history/10765/ranking/687e5ab2416a2ae510ec19ff40921d7e?stage=11&type=etg

Stage number: 12
https://www.letour.fr/en/block/hist

...

https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=9&type=ijg

Stage number: 10
https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=10&type=ijg

Stage number: 11
https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=11&type=ijg

Stage number: 12
https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=12&type=ijg

Stage number: 13
https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=13&type=ijg

Stage number: 14
https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=14&type=ijg

Stage number: 15
https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=15&type=ijg

Stage number: 16
https://www.letour.fr/en/block/history/10760/ranking/09063ac0091cd75ca4a3c7001f4ba2f4?stage=16&type=ijg

Stage number: 17
https://www.letour.fr/en/

https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=7&type=itg

Stage number: 8
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=8&type=itg

Stage number: 9
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=9&type=itg

Stage number: 10
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=10&type=itg

Stage number: 11
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=11&type=itg

Stage number: 12
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=12&type=itg

Stage number: 13
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=13&type=itg

Stage number: 14
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=14&type=itg

Stage number: 15
https://www.letour.fr/en/bloc

https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=8&type=img

Stage number: 9
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=9&type=img

Stage number: 10
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=10&type=img

Stage number: 11
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=11&type=img

Stage number: 12
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=12&type=img

Stage number: 13
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=13&type=img

Stage number: 14
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=14&type=img

Stage number: 15
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=15&type=img

Stage number: 16
https://www.letour.fr/en/bl

https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=9&type=etg

Stage number: 10
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=10&type=etg

Stage number: 11
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=11&type=etg

Stage number: 12
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=12&type=etg

Stage number: 13
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=13&type=etg

Stage number: 14
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=14&type=etg

Stage number: 15
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=15&type=etg

Stage number: 16
https://www.letour.fr/en/block/history/10759/ranking/a6d2a4b119f62d1c9de9fc05047e5552?stage=16&type=etg

Stage number: 17
https://www.letour.fr/en/

ValueError: Length mismatch: Expected axis has 3 elements, new values have 5 elements